In [2]:
!pip install pandas

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.6 MB 3.3 MB/s eta 0:00:04
   ------ --------------------------------- 1.8/11.6 MB 3.5 MB/s eta 0:00:03
   --------- ------------------------------ 2.6/11.6 MB 3.4 MB/s eta 0:00:03
   ---------- ----------------------------- 3.1/11.6 MB 3.3 MB/s eta 0:00:03
   ------------- -------------------------- 3.9/11.6 MB 3.4 MB/s eta 0:00:03
   ---------------- ----------------------- 4.7/11.6 MB 3.4 MB/s eta 0:00:03
   ------------------ --------------------- 5.5/11.6 MB 3.4 MB/s eta 0:00:02
   -------------------- ------------------- 6.0/11.6 MB 3.4 MB/s eta 0:00:02
   ----------------------- ---------------- 6.8/11.6 MB 3.4 MB/s eta 0:00:02
   ------------------------- -----

In [5]:
!pip install scikit-learn
!pip install matplotlib

  Using cached joblib-1.5.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 5.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.3/11.1 MB 3.5 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.1 MB 3.4 MB/s eta 0:00:03
   --------- ------------------------------ 2.6/11.1 MB 3.4 MB/s eta 0:00:03
   ------------ --------------------------- 3.4/11.1 MB 3.4 MB/s eta 0:00:03
   -------------- ------------------------- 3.9/11.1 MB 3.4 MB/s eta 0:00:03
   ---------------- ----------------------- 4.7/11.1 MB 3.4 MB/s eta 0:00:02
   ------------------- -------------------- 5.5/11.1 MB 3.4 MB/s eta 0:00:02
   --------------------- ------------------ 6.0/11.1 MB 3.4 MB/s eta 0:00:02
   ------------------------ --------------- 6.8/11.1 MB 3.4 MB/s eta 0:00:02
   -------------------------

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

# Simply about Word2Vec (Quora dataset)

***Liana Napalkova***

***21 October 2018***

## Table of contents
1. [Introduction](#introduction)
2. [What is Word2Vec?](#word2vec)
3. [Training of Word2Vec](#training)
4. [Exploring the trained Word2Vec model](#exploring)
5. [Playing with the trained Word2Vec model ](#playing)
6. [When can we use Word2Vec?](#usage)

**I hope you find this kernel helpful and some <font color="red"><b>UPVOTES</b></font> would be very much appreciated.**

## Introduction <a name="introduction"></a>

If you have never used Word2Vec, then this notebook is for you! Here I will apply Word2Vec to the questions asked in Quora. The goal of the notebook is to better understand how Word2Vec works. Basically I wanted to resolve my personal doubts regarding Word2Vec. I will be very glad if my small analysis would be useful for someone else.

In [8]:
df = pd.read_csv("q_quora.csv")
df.head()

C:\Users\SYST\AppData\Local\Temp\ipykernel_5400\1610249523.py:1: DtypeWarning: Columns (7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("q_quora.csv")


,id,qid1,qid2,question1,question2,is_duplicate,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.shape

(404351, 13)

The Quora dataset consists of 404.290 rows and 6 columns. To reach the goal of this notebook, I am only interested in columns that contain questions. These columns are "question1" and "question2". To reduce training time, I will only use the column "question1".

## What is Word2Vec? <a name="word2vec"></a>

Word2Vec is one of many different word embedding techniques. In turn, word embedding is one of the most popular representation of document vocabulary. Thus, we get the following hierarchy:

* Document vocabulary representation -> Word embedding -> Word2Vec

So, what is word embedding? **Word embedding** is a vector representation of a word. For example, let's consider the folling three phases: 
* Agama is a reptile.
* Snake is a reptile.
* Reptile is a cold-blooded animal.

The **dictionary** of this corpus of text (three phrases) consists of all unique words:
> ["agama", "reptile", "snake", "a", "is", "cold-blooded", "animal"]

A simple vector representation of the word "agama" would be:
> [1, 0, 0, 0, 0, 0, 0]

where 1 stands for the word "agama" in our dictionary. This word embedding is called one-hot encoding. For any word, the length of a vector is always 7 in our example, because the dictionary consists of 7 unique words. This word embedding technique is very simple, but has a tremendous disadvantage. Let's take vector representations of two words: 
> "agama" and "reptile" -> [1, 0, 0, 0, 0, 0, 0] and [0, 1, 0, 0, 0, 0, 0]

These two vectors provide no useful information regarding the relationships that may exist between the "agama" and "reptile". In other words, the similarity between words is not captured by the one-hot encoding. Loosely speaking, one-hot encoding does not enable the system making conclusions about the word "agama" given the word "reptile" (for example, such that agama is a cold-blooded animal).

**Word2Vec is a more sophisticated word embedding technique**. This technique is based on the idea that words that occur in the same **contexts** tend to have similar meanings. I like this definition from Wikipedia 

> Word2Vec is a **two-layer neural network** that takes as its input a large corpus of text and produces a vector space, typically of several hundred dimensions, with each unique word in the corpus being assigned a corresponding vector in the space (Wikipedia).

Below I provide one of the best graphic representations of Word2Vec's word embedding that I found on Internet. This picture gives an intuition about how word embedding looks like in Word2Vec.

![](http://adriancolyer.files.wordpress.com/2016/04/word2vec-distributed-representation.png?w=566&zoom=2)

Ref: [The amazing power of word vectors](http://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/)

**Word2Vec itself is not a deep neural network**, but it turns input text into a numerical form that deep neural networks can process as inputs.

## Training of Word2Vec <a name="training"></a>

To train the Word2Vec two-layer neural network, the only thing that we need to do is to change the format of Quora dataset. In particular, I will transform the column "question1" into a list of lists. I will use the function "simple_preprocess" of GenSim package in order to transform each question (row) into a set of tokens, for example: "what is your name" -> "what", "is", "your", "name". 

In [ ]:
def read_questions(row,column_name):
    return gensim.utils.simple_preprocess(str(row[column_name]).encode('utf-8'))
    
documents = []
for index, row in df.iterrows():
    documents.append(read_questions(row,"question1"))
    if row["is_duplicate"] == 0:
        documents.append(read_questions(row,"question2"))1

In [16]:
def read_questions(row, column_name):
    return gensim.utils.simple_preprocess(str(row[column_name]))

In [17]:
documents = []
for index, row in df.iterrows():
    if pd.notnull(row["question1"]):  # Vérifiez que la question n'est pas NaN
        documents.append(read_questions(row, "question1"))
    if row["is_duplicate"] == 0 and pd.notnull(row["question2"]):  # Vérifiez aussi pour question2
        documents.append(read_questions(row, "question2"))

In [18]:
print(type(documents))  # Doit être une liste
print(type(documents[0]))  # Doit être une liste
print(documents[:2])  # Affiche les deux premières entrées

<class 'list'>
<class 'list'>
[['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india'], ['what', 'is', 'the', 'story', 'of', 'kohinoor', 'koh', 'noor', 'diamond']]


In [11]:
print("List of lists. Let's confirm: ", type(documents), " of ", type(documents[0]))

List of lists. Let's confirm:  <class 'list'>  of  <class 'list'>


Now Word2Vec will retrieve all unique words from all sub-lists of "documents", thereby constructing the vocabulary (unique words). 

In [19]:
# Remplacez 'size' par 'vector_size'
model = gensim.models.Word2Vec(vector_size=150, window=10, min_count=2, sg=1, workers=10)
model.build_vocab(documents)  # Préparer le vocabulaire du modèle

Below I provide the definition of four parameters that we used to define a Word2Vec model:

* **size:** The size means the dimensionality of word vectors. It defines the number of tokens used to represent each word. For example, rake a look at the picture above. The size would be equal to 4 in this example. Each input word would be represented by 4 tokens: King, Queen, Women, Princess.  Rule-of-thumb: If a dataset is small, then size should be small too. If a dataset is large, then size should be greater too. It's the question of tuning.

* **window:** The maximum distance between the target word and its neighboring word. For example, let's take the phrase "agama is a reptile " with 4 words (suppose that we do not exclude the stop words). If window size is 2, then the vector of word "agama" is directly affected by the word "is" and "a". Rule-of-thumb: a smaller window should provide terms that are more related (of course, the exclusion of stop words should be considered).

* **min_count:** Ignores all words with total frequency lower than this. For example, if the word frequency is extremally low, then this word might be considered as unimportant.

* **sg:** Selects training algorithm: 1 for Skip-Gram; 0 for CBOW (Continuous Bag of Words).

* **workers:** The number of worker threads used to train the model.

More details about input parameters can be found [here](http://radimrehurek.com/gensim/models/word2vec.html).

Once the vocabulary is created and the Word2Vec model is specified, I will train this model by calling "train" function.

In [22]:

model.train(sentences=documents, total_examples=len(documents), epochs=3)

TypeError: Either one of corpus_file or corpus_iterable value must be provided

Let's see what happens when the function "train" is executed. I found a diagram (see below) that shows a Skip-Gram model architecture to produce a distributed representation of words. To understand this diagram, we should consider the size of vocablurary and the dimensionality of the word vectors (the parameter "size"). In our example, these are equal to 47.366 words and 150 tokens, respectively. If we take the word “immigration”, it will be one of the words in the 47.366 word vocabulary.  Therefore we can initially represent the word "immigration" as a 47.366 length one-hot vector.  We then link this input vector to a 150 node hidden layer.  There are totally 150 weights connecting the input layer and the hidden layer.  The activations of the nodes in this hidden layer are simply linear summations of the weighted inputs, e.g. `ohe*w1+ohe*w2+...+ohe*w150`.  These nodes are then fed into a softmax output layer.  During training, we want to change the weights of this neural network so that words surrounding “immigration” have a higher probability in the softmax output layer.  So, for instance, if our text data set has a lot of questions related to the immigration in Canada, we would want our network to assign large probabilities to words like “Canada”, "Australia"  (given lots of sentences containing “the immigration to Canada or Australia”).

![](https://preview.ibb.co/iuzji0/Word2-Vec-skip-gram.jpg)

We finished training our Word2Vec model. What's next? The funny thing is that next we abandon the softmax layer and just use the 47.366 x 150 weight matrix as our word embedding lookup table. Sounds weird, isn't it? The only goal was to get the weights of the hidden layer. These weights are essentially the word embeddings that we had to learn. Once we know these word embeddings, we do not need the whole Word2Vec network anymore.

## Exploring the trained Word2Vec model <a name="exploring"></a>

The learned vocabulary of tokens (words) is stored in "model.wv.vocab". In our case, the vocabulary includes 47.366 unique words. 

In [23]:
word_vectors = model.wv
count = 0
for word in word_vectors.vocab:
    if count<10:
        print(word)
        count += 1
    else:
        break

AttributeError: The vocab attribute was removed from KeyedVector in Gensim 4.0.0.
Use KeyedVector's .key_to_index dict, .index_to_key list, and methods .get_vecattr(key, attr) and .set_vecattr(key, attr, new_val) instead.
See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4

In [ ]:
len(word_vectors.vocab)

The embedded vectors for a specific token are stored in a KeyedVectors instance in "model.wv". We can see that the length of a word vector is equal to 150 as we defined by the parameter "size":

In [ ]:
vector = model.wv["immigration"]  # numpy vector of a word
len(vector)

In [ ]:
vector

We can now create a two-dimensional semantic representation of word embeddings using t-SNE.

In [ ]:
wanted_words = []
count = 0
for word in word_vectors.vocab:
    if count<150:
        wanted_words.append(word)
        count += 1
    else:
        break
wanted_vocab = dict((k, word_vectors.vocab[k]) for k in wanted_words if k in word_vectors.vocab)
wanted_vocab

In [ ]:
X = model[wanted_vocab] # X is an array of word vectors, each vector containing 150 tokens
tsne_model = TSNE(perplexity=40, n_components=2, init="pca", n_iter=5000, random_state=23)
Y = tsne_model.fit_transform(X)

In [ ]:
#Plot the t-SNE output
fig, ax = plt.subplots(figsize=(20,10))
ax.scatter(Y[:, 0], Y[:, 1])
words = list(wanted_vocab)
for i, word in enumerate(words):
    plt.annotate(word, xy=(Y[i, 0], Y[i, 1]))
ax.set_yticklabels([]) #Hide ticks
ax.set_xticklabels([]) #Hide ticks
_ = plt.show()

Not that bad. It's interesting to see that, for example, the word pairs "motorola" and "phone" or "youtube" and "video" are located quite close to each other. It means that these words appeared in the same context in the corpus of text.

## Playing with the trained Word2Vec model <a name="playing"></a>

Now we will see what we can actually do with our trained Word2Vec model. This overview does not pretend to be exhaustive. I will only check the most popular functionalities:
* Given a word, find most similar words
* Given words A and B, find other words that are similar to A and opposite to B
* Given a sequence of words, find an odd word

This first example shows a simple case of looking up top 5 words similar to the word "phone". To do this, I will call the "most_similar" function and provide the word "phone" as the positive example. This returns the top 5 similar words: "mobile", "phones", "smartphone", "iphone" and "device". 

You may notice that there is no word "motorola" in the list of top 5 similar words, while the word "motorola" appears in t-SNE graphic. Please remember that t-SNE was applied on a small subset of vocabulary (150 words). Therefore, the words "mobile", "phones", "smartphone", etc. were simply not included in these 150 words and therefore they have not appeared on the t-SNE graphic.

In [ ]:
w1 = "phone"
model.wv.most_similar(positive=w1, topn=5)

Next, given positive and negative words, we will find top 2 words that are similar to positivie words and opposite to negative words.

In [ ]:
w1 = ["women","rights"]
w2 = ["girls"]
model.wv.most_similar (positive=w1,negative=w2,topn=2)

Finally, I will use Word2Vec to find odd items given a list of items "government","corruption" and "peace". For most of people this is a rhetorical question.

In [ ]:
model.wv.doesnt_match(["government","corruption","peace"])

Do you agree with the opinion of Word2Vec?:)


## When can we use Word2Vec? <a name="usage"></a>

There are many application scenarios for Word2Vec: sentiment analysis, recommender systems, etc. For example, imagine that you have tags for a large amount of questions in Quora. Given a tag, you want to find similar tags in order to recommend them to a user for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. 